# MVP 1 - User Interface using Pythonic code

In [11]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
import pandas as pd
import requests
from eventregistry import *
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [17]:
!pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [12]:
load_dotenv()

True

In [13]:
@tool
def stock_prices_tool(stock_name):

    """Function to fetch stock prices from Alpha Vantage"""

    alpha_vantage_api_key = os.getenv('ALPHA_VANTAGE_API_KEY')

    try:
        url = f'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={stock_name}&apikey={alpha_vantage_api_key}'
        r = requests.get(url)
        data = r.json()
    except Exception as e:
        print(f"Alpha Vantage API Error occurred: \n {e}")
        

    stock_ticker = ''
    for match in data['bestMatches']:
        if 'BSE' in match['1. symbol']:
           stock_ticker = match['1. symbol']
    
    if stock_ticker == '':
        raise ValueError('Stock not found in BSE')

    try:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock_ticker}&apikey={alpha_vantage_api_key}'
        r = requests.get(url)
        data = r.json()
    except Exception as e:
        print(f"Alpha Vantage API Error occurred: \n {e}")
    
    #print(len(data['Time Series (Daily)']))
    
    stock_prices_json = data['Time Series (Daily)']

    stock_prices_df = pd.DataFrame(stock_prices_json).T
    stock_prices_df.rename(columns={'1. open':'open','2. high':'high','3. low':'low','4. close':'close','5. volume':'volume'},inplace=True)
    stock_prices_df.reset_index(inplace=True)
    stock_prices_df.rename(columns={'index':'date'},inplace=True)

    stock_prices_mkd = stock_prices_df.to_markdown(index=False)

    return stock_prices_mkd

In [14]:
@tool
def financial_news_tool(stock_name,newsapi_api_key):

    """Function to fetch financial news from NewsAPI"""

    newsapi_api_key = os.getenv("NEWSAPI_API_KEY")
    try:
        er = EventRegistry(apiKey = newsapi_api_key)
        query = {
          "$query": {
            "$and": [
              {
                "keyword": stock_name,
                "keywordLoc": "title"
              },
              {
                "locationUri": "http://en.wikipedia.org/wiki/India"
              },
              {
                "lang": "eng"
              }
            ]
          },
          "$filter": {
            "forceMaxDataTimeWindow": "31"
          }
        }
        q = QueryArticlesIter.initWithComplexQuery(query)
        article_list = []
        # change maxItems to get the number of results that you want
        for i,article in enumerate(q.execQuery(er, maxItems=5)):
            #print(f'Article {i+1}')
            #print(article['title'])
            #print(article['body'])
            title = article['title']
            body = article['body']
            article = '\n\n'.join([title,body])
            article_list.append(article)
    
        article_string = '\n\n\n'.join([f"**Article {i+1}**:\n\n{article}" for i,article in enumerate(article_list)])
    
        return article_string
    except Exception as e:
        print(f"NewsAPI API Error occurred: \n {e}")

In [18]:
def stock_recommendation_agent_mvp1():
    stock_name = input("Enter the stock that you want to get buy or sell recommendation")
    
    #alpha_vantage_api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
    #newsapi_api_key = os.getenv("NEWSAPI_API_KEY")
    openai_endpoint = os.getenv("OPENAI_API_ENDPOINT")
    openai_api_key = os.getenv("OPENAI_API_KEY")
    openai_deployment = os.getenv("OPENAI_API_DEPLOYMENT")
    openai_version = os.getenv("OPENAI_API_VERSION")

    llm = AzureChatOpenAI(
            openai_api_version=openai_version,
            azure_endpoint=openai_endpoint,
            openai_api_key=openai_api_key
        )

    messages = [
    {"role": "system", "content": """You are a financial advisor capable to 
    recommending stocks to buy or sell. Your task is to fetch five articles and last 100 days stock prices regarding {stock_name} using the tools 
    and analyze the articles to predict whether the stock prices of {stock_name} will move in a bullish or 
    bearish manner and give recommendation on whether to buy or sell stock."""},
    MessagesPlaceholder("agent_scratchpad")
    ]
    
    prompt = ChatPromptTemplate.from_messages(messages)

    tools = [stock_prices_tool,financial_news_tool]

    llm_with_tools = llm.bind_tools(tools)
    
    agent = create_tool_calling_agent(llm_with_tools, tools, prompt)

    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
    ai_msg = agent_executor.invoke({"stock_name":stock_name})

    return ai_msg
    

In [19]:
output = stock_recommendation_agent_mvp1()

Enter the stock that you want to get buy or sell recommendation SBIN




> Entering new AgentExecutor chain...

Invoking: `stock_prices_tool` with `{'stock_name': 'SBIN'}`


| date       |   open |   high |    low |   close |   volume |
|:-----------|-------:|-------:|-------:|--------:|---------:|
| 2025-11-10 | 955.95 | 963    | 949.15 |  951.85 |   387727 |
| 2025-11-07 | 959.95 | 964.5  | 944    |  955.95 |   582333 |
| 2025-11-06 | 966.35 | 971.15 | 955.3  |  960.75 |  1731058 |
| 2025-11-04 | 952.25 | 959    | 941.35 |  957.05 |   948033 |
| 2025-11-03 | 938.5  | 953.15 | 937.8  |  950.25 |   696733 |
| 2025-10-31 | 935.8  | 946.7  | 930.25 |  937    |   780748 |
| 2025-10-30 | 940    | 944.6  | 932.9  |  934.1  |  1338039 |
| 2025-10-29 | 932.6  | 941.65 | 930.3  |  939.8  |  1089247 |
| 2025-10-28 | 923.25 | 935.9  | 917.8  |  930.25 |   641738 |
| 2025-10-27 | 905.5  | 927.9  | 905.5  |  923.25 |   392249 |
| 2025-10-24 | 911.45 | 914.2  | 900.8  |  904.4  |   176431 |
| 2025-10-23 | 910.55 | 917.9  | 904.8  |  911.45 |  1321998 |
| 2025-10-20 | 